# 欠損値を穴埋めする関数を作成

## 条件

- 欠損箇所を埋める既定値には列ごとにある
- データはintとflortの場合がある
- 列内のNoneのところだけに穴埋め処理し、他は影響なし
- 空文字はNone扱い
- 既定値の値もデータ型もそのまま変えずに穴埋め
- 穴埋めの既定値を基本的にint型として、float型で穴埋めしたい列名を引数でリスト形式で指定できる

## 困りごと

pd.fillna(var)を使うと、以下の機能が実現できない。

- 欠損値を既定値で埋めるけど、他の値のデータ型は壊したくない。
- 列ごとに別の値で埋めたい。
- NaNは無視したい。




## 1. 入力：データフレームを作成

In [2]:
# 欠損値の穴埋め
import numpy as np
import pandas as pd
dfm = pd.DataFrame(
    [
    {"hash":10,"age":28,"wt":56.1},
    {"hash":20,"age":None,"wt":None},
    {"hash":30,"age":'',"wt":''},
    {"hash":40,"name":33,"wt":100}
    ],columns=["hash","age","wt"]
)

dfm



,hash,age,wt
0,10,28,56.1
1,20,None,None
2,30,,
3,40,NaN,100


## 2. 処理：データ型を変えずに穴埋め

1. データ型を保持しておくデータフレームを作成
2. データをNoneにのみ代入（NaNは無視）
3. データ型が代入した値に応じて列ごとに変化してしまうので、

In [5]:
# 1. データ型を保持しておくデータフレームを作成
dfm_type = dfm.copy()

def get_type(val):
    return type(val)

dfm_type = dfm_type.applymap(get_type)

dfm_type

,hash,age,wt
0,<class 'int'>,<class 'int'>,<class 'float'>
1,<class 'int'>,<class 'NoneType'>,<class 'NoneType'>
2,<class 'int'>,<class 'str'>,<class 'str'>
3,<class 'int'>,<class 'float'>,<class 'int'>


In [6]:
# 2. データ型を確認し、'NoneType'の場合のみ代入
def fill_none(dfm, col_name, default_val):
    DEFAULT = default_val

    def set_default(val):
        if isinstance(val, type(None)):
            return DEFAULT
        else:
            return val    

    # 欠損値（None）を穴埋め
    dfm[col_name] = dfm[col_name].map(set_default)
    
    return dfm

# 3. 保持しておいたデータ型を適用
def set_type(dfm, dfm_type, col_name, default_val):
    for i in range(len(dfm_type)):
        pre_type = dfm_type.loc[i, col_name]
        if pre_type == type(None):
            if isinstance(default_val, int):
                dfm.loc[i, col_name] = int(dfm.loc[i, col_name])
            elif isinstance(default_val, float):
                dfm.loc[i, col_name] = float(dfm.loc[i, col_name])
        elif pre_type == int:
            dfm.loc[i, col_name] = int(dfm.loc[i, col_name])
        elif pre_type == float:
            dfm.loc[i, col_name] = float(dfm.loc[i, col_name])
            
    return dfm


# 既定値を設定
AGE_DEFAULT = 99 # int型
WT_DEFAULT = 99.9 # float型

# 列名と既定値のペアを辞書型で作成
default_val_dict = {'age': AGE_DEFAULT, 'wt': WT_DEFAULT}

# 空文字をNoneに置換
dfm = dfm.replace('', None)

# データ型が自動で変更されないように、一時的にstr型の行を追加する
dfm.loc[len(dfm)] = ['tmp'] * len(dfm.columns)

# データ型に影響を与えずに欠損値の穴埋めを行う
for col_name, default_val in default_val_dict.items():
    dfm = fill_none(dfm, col_name, default_val)
    set_type(dfm, dfm_type, col_name, default_val)

# データ型が自動で変更されないように追加した行を削除
dfm = dfm.drop(len(dfm)-1, axis=0)
      
dfm

,hash,age,wt
0,10,28,56.1
1,20,99,99.9
2,30,99,99.9
3,40,NaN,100


## 3. 出力：各データのデータ型を確認

In [10]:
for col in dfm.columns:
    print('[{}]=================='.format(col))
    print(dfm[col].map(type))
    print()

dfm

[hash]==================
0    <class 'int'>
1    <class 'int'>
2    <class 'int'>
3    <class 'int'>
Name: hash, dtype: object

[age]==================
0      <class 'int'>
1      <class 'int'>
2      <class 'int'>
3    <class 'float'>
Name: age, dtype: object

[wt]==================
0    <class 'float'>
1    <class 'float'>
2    <class 'float'>
3      <class 'int'>
Name: wt, dtype: object



,hash,age,wt
0,10,28,56.1
1,20,99,99.9
2,30,99,99.9
3,40,NaN,100


## TODO：機能を一般化

---

In [4]:
print(type(''))
print(type(None))
print(type(np.nan))

<class 'str'>
<class 'NoneType'>
<class 'float'>


In [7]:
x_int = 99
type(x_int)

int

In [8]:
x_float = float(x_int)
type(x_float)

float

In [9]:
x_float

99.0

In [10]:
type(type(x_float))

type

In [9]:
# 欠損箇所を埋める既定値には列ごとにある。データはintとflortの場合がある
# これはダメなケース
# AGE_DEFAULT = 99
# WT_DEFAULT = 99.9

dfm = dfm.fillna(99)

dfm

,hash,age,wt
0,10,28,56.1
1,20,99,99.0
2,30,99,99.0
3,40,99,100.0


In [39]:
# 2. 列ごとに異なる値で穴埋め
# https://note.nkmk.me/python-pandas-nan-dropna-fillna/

dfm = dfm.fillna({
    'age': AGE_DEFAULT, 
    'wt': WT_DEFAULT
    })

dfm

,hash,age,wt
0,10,28,56.1
1,20,99,99.9
2,30,99,99.9
3,40,99,100.0


In [ ]:
# 仕様検討中
# 　既定値の値もデータ型もそのまま変えずに穴埋めをしたい。
# 　穴埋めの既定値を基本的にint型として、float型で穴埋めしたい列名はリストにして引数で指定する
#　列内のNoneのところだけに穴埋め処理し、他は影響しないようにしたい
